In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
import os
import cv2
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose,concatenate
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import StandardScaler
from keras.utils.data_utils import get_file

Using TensorFlow backend.


In [0]:
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import math
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
import zipfile

zipfilePath = ("Slow.zip")
zip = zipfile.ZipFile(zipfilePath)
zip.extractall(".")
zip.close()

FileNotFoundError: ignored

In [0]:
def get_file(file, files, bound = 20):
    if(os.path.isdir(file)):
        for fi in sorted(os.listdir(file)):
            get_file('{}/{}'.format(file, fi), files)
    else:
        basename = os.path.basename(file)
        value = int(basename[:-4])
        
        if value < 20:
            files.append(file)

In [0]:
input_folder = 'Slow/Blurred'
output_folder = 'Slow/Original'


In [0]:
input_files = list()
get_file(input_folder, input_files)

output_files= list()
get_file(output_folder, output_files)

In [0]:
def getdata(files, batch_size = 20):
    data = list()
    for i in range(0, len(files), batch_size):
        datum = []
        for ii in range(i, i+batch_size):
            filepath = files[ii]
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            datum.append(img)
        data.append(datum)
    return data;

In [0]:
X = getdata(input_files)
X = np.array(X)

y = getdata(output_files)
y = np.array(y)

print('Input Shape: {}'.format(X.shape))
print('Output Shape: {}'.format(y.shape))

In [0]:
cnt = 130
X = X[:cnt]
y = y[:cnt]

l = list(range(0, cnt))
np.random.shuffle(l)
X = X[l]
y = y[l]

In [0]:
x_train, y_train = X[:90], y[:90]
x_val, y_val = X[90:110], y[90: 110]
x_test, y_test = X[110:], y[110:]
x_train.shape


In [0]:
m = np.max(X)
mi = np.min(X)
print(m, mi)

In [0]:
X = (X - mi) / (m - mi)
m1 = np.max(X)
mi1 = np.min(X)
print(m1, mi1)


In [0]:
a = np.max(y)
b = np.min(y)
y = (y - b) / (a - b)
print(a, b)

In [0]:
#Setting hyperparameters which may change according to dataset

batch_size = 16
epochs = 220
inChannel = 20
x, y = 256, 256
input_img = Input(shape = (x, y, inChannel))

In [0]:
def autoencoder(input_tensor):
    #input_tensor = Input(shape=(256,256,20,))
    x = Conv2D(filters = 64, kernel_size=3, padding = 'same', activation='relu')(input_tensor)
    x = BatchNormalization()(x)
    x = Conv2D(filters = 64, kernel_size=3, padding = 'same', activation='relu')(x)
    x = BatchNormalization()(x)

    y = x

    y = MaxPooling2D(strides = 2)(y)
    y = Conv2D(filters = 128, kernel_size=3, padding = 'same', activation='relu')(y)
    y = BatchNormalization()(y)
    y = Conv2D(filters = 128, kernel_size=3, padding = 'same', activation='relu')(y)
    y = BatchNormalization()(y)


    z = y

    z = MaxPooling2D(strides = 2)(z)
    z = Conv2D(filters = 256, kernel_size=3, padding = 'same', activation='relu')(z)
    z = BatchNormalization()(z)
    z = Conv2D(filters = 256, kernel_size=3, padding = 'same', activation='relu')(z)
    z = BatchNormalization()(z)


    w = z

    w = MaxPooling2D(strides = 2)(w)
    w = Conv2D(filters = 512, kernel_size=3, padding = 'same', activation='relu')(w)
    w = BatchNormalization()(w)
    w = Conv2D(filters = 512, kernel_size=3, padding = 'same', activation='relu')(w)
    w = BatchNormalization()(w)

    u = w

    u = MaxPooling2D(strides = 2)(u)
    u = Conv2D(filters = 1024, kernel_size=3, padding = 'same', activation='relu')(u)
    u = BatchNormalization()(u)
    u = Conv2D(filters = 1024, kernel_size=3, padding = 'same', activation='relu')(u)
    u = BatchNormalization()(u)
    u = Conv2DTranspose(filters = 512, kernel_size = 1, strides = 2)(u)

    v = concatenate([w, u])
    v = Conv2D(filters = 512, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2D(filters = 512, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2DTranspose(filters = 256, kernel_size = 1, strides = 2)(v)


    v = concatenate([z, v])
    v = Conv2D(filters = 256, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2D(filters = 256, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2DTranspose(filters = 128, kernel_size = 1, strides = 2)(v)


    v = concatenate([y, v])
    v = Conv2D(filters = 128, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2D(filters = 128, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2DTranspose(filters = 64, kernel_size = 1, strides = 2)(v)


    v = concatenate([x, v])
    v = Conv2D(filters = 64, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2D(filters = 64, kernel_size=3, padding = 'same', activation='relu')(v)
    v = BatchNormalization()(v)
    v = Conv2D(filters = 20, kernel_size = 1, strides = 1)(v)
    
    return v

In [0]:
#compiling using optimizer and loss function is defined

autoencoder = Model(input_img, autoencoder(input_img))
autoencoder.compile(loss='mean_squared_error', optimizer = Adam() )

In [0]:
autoencoder.summary()

In [0]:
x_temp = x_train.reshape(-1 , 256, 256, 20)

In [0]:
x_temp.shape

In [0]:
y_temp = y_train.reshape(-1, 256, 256, 20)
x_tempval = x_val.reshape(-1, 256, 256, 20)
y_tempval = y_val.reshape(-1, 256, 256, 20)

In [0]:
#Model is trained
autoencoder_train = autoencoder.fit(x_temp, y_temp, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_tempval, y_tempval))

In [0]:
#loss plot between training and validation data to visualise the model performance.

loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(220)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [0]:
#Predicting on validation data
pred = autoencoder.predict(x_tempval)

In [0]:
pred = pred.reshape(-1, 20, 256, 256) * 256
y_tempval = y_tempval.reshape(-1, 20, 256, 256) * 256

In [0]:
plt.figure(figsize=(20, 4))
print("Reconstruction of Test Images")
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(y_tempval[i][1], cmap='gray')
plt.show()    
plt.figure(figsize=(20, 4))
print(" Test Images")
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(pred[i][1], cmap='gray')  
plt.show()

In [0]:
plt.figure(figsize=(20, 4))
print("Reconstruction of Test Images")
plt.subplot(1,4,1)
plt.imshow(y_tempval[2][1], cmap='gray')
plt.subplot(1,4,2)
plt.imshow(y_tempval[3][1], cmap='gray')
plt.subplot(1,4,3)
plt.imshow(y_tempval[5][1], cmap='gray')
plt.subplot(1,4,4)
plt.imshow(y_tempval[8][1], cmap='gray')
plt.show()    
plt.figure(figsize=(20, 4))
print(" Test Images")
plt.subplot(1,4,1)
plt.imshow(pred[2][1], cmap='gray')  
plt.subplot(1,4,2)
plt.imshow(pred[3][1], cmap='gray')
plt.subplot(1,4,3)
plt.imshow(pred[5][1], cmap='gray')
plt.subplot(1,4,4)
plt.imshow(pred[8][1], cmap='gray')
plt.show()